In [ ]:
import numpy as np
import walnut
from walnut import Tensor

In [ ]:
with open("data/tinyshakespeare.txt", "r") as f:
    data = "".join(f.readlines())

data_red = data.lower()

In [ ]:
i_to_s = {i:s for i,s in enumerate(sorted(set(data_red)))}
s_to_i = {s:i for i,s in enumerate(sorted(set(data_red)))}
vocab_size = len(i_to_s)

In [ ]:
num_samples = 10000
rand_indices = np.random.randint(0, len(data_red) - 1, (num_samples,))

In [ ]:
# convert array of characters to indices
x_indices = [s_to_i[data_red[i]] for i in rand_indices]
y_indices = [s_to_i[data_red[i + 1]] for i in rand_indices]

# initialize Tensors and reshape
X = Tensor(x_indices)
Y = Tensor(y_indices)

print(f"{X.shape=}")
print(f"{Y.shape=}")

In [ ]:
X_enc = walnut.preprocessing.one_hot_encode(X, vocab_size)
Y_enc = walnut.preprocessing.one_hot_encode(Y, vocab_size)

print(f"{X_enc.shape=}")
print(f"{Y_enc.shape=}")

In [ ]:
import walnut.nn as nn

model = nn.Sequential(layers=[
    nn.layers.Character(10, input_shape=(vocab_size,)),
    nn.layers.Linear(100, act="tanh", norm="layer"),
    nn.layers.Linear(vocab_size, act="softmax")
])

In [ ]:
model.compile(nn.optimizers.Adam(), nn.losses.Crossentropy(), nn.metrics.Accuracy())

In [ ]:
model

In [ ]:
# high score 0.0614
train_hist, val_hist = model.train(X_enc, Y_enc, epochs=5000, verbose="reduced")

In [ ]:
traces = {
    "train_loss" : train_hist,
    "val_loss" : val_hist
}
nn.analysis.plot_curve(traces=traces, figsize=(20, 4), title="loss history", x_label="epoch", y_label="loss")

In [ ]:
p_char = Tensor([1]) # use " " as start character
p_char = walnut.preprocessing.one_hot_encode(p_char, vocab_size)

for i in range(1000):
    pred = model(p_char) # get model prediction for a character
    index = walnut.choice(pred) # choose following character
    print(i_to_s[index], end="")
    p_char = walnut.preprocessing.one_hot_encode(Tensor([index]), vocab_size) # set base character to be predicted character